In [ ]:
!jt -t chesterish

In [6]:
import numpy as np
import scipy.spatial.distance

In [142]:
from _entries import RoutingEntry, GroundEntry
from _nodes import Root, Leaf, Router
from heuristics import *

class MtreeNode:
    def __init__(self):
        self.height = 0
        self.id = 0
        self.points = []
        self.children = []
        self.parent = None
        


class MTree:
    """
    Represents M-Tree data structure
    """

    def __init__(self, capacity_max: int = 9, dist_function=dist_euclidean, split_function=split_data_random):
        """
        :param capacity_max: maximal number of objects any node can store
        :param dist_function: metrics used to determine distance of 2 data objects, default is euclidean distance
        has to take two parameters (data) and return distance between them
        :param split_function: split heuristics function, default split heuristics is random split
        has to take dictionary and return two data partitions
        """
        self._root = None
        self.capacity_min = 2
        self.capacity_max = capacity_max
        self.split_function = split_function
        self._dist_function = dist_function
        self.rootNode  = None
        self.documentMap = {}
        
    
        
        

    def __str__(self):
        # just display the root
        return f'M-Tree: < {self._root} >'

    def add(self, data):
        """
        Adds new data into the M-Tree
        :param data: data to be inserted
        :return: success
        """
        # tree might be empty
        if self._root is None:
            # init the root with first entry
            self._init_root(data)
            return True
        else:
            # try to add data to the existing root node
            if self._root.add(data):
                # check root capacity
                if self._root.is_overflowed(max_capacity=self.capacity_max):
                    # add one level to the tree
                    self._split_root()
                return True
            # couldn't add data
            return False

    def delete(self, data):
        """
        Removes an object containing passed data from the M-Tree
        :param data: data to be removed
        :return: success
        """
        # tree might be empty
        if self._root is None:
            return False
        # try to delete the data
        if self._root.delete(data):
            if self._root.is_underflowed(min_capacity=1):
                self._root = None
            return True
        # couldn't delete data
        return False

    def range_query(self, data, r):
        """
        Finds all object within the range from the data object
        :param data: query object data
        :param r: query range
        :return: list of r-similar objects
        """
        # tree might be empty
        if self._root is None:
            return []
        # count distance to the root
        d = self._dist_function(data, self._root.data)
        # simply run search from the root
        return self._root.search(data=data, d_parent=d, r=r)

    def knn_query(self, data, k):
        """
        Finds k objects closest to the queried one
        :param data: query object data
        :param k: number of closest neighbours to be found
        :return: list of k (or less, in case there is not enough objects) most similar objects
        """
        # tree might be empty
        if self._root is None:
            return []
        # count distance to the root
        d = self._dist_function(data, self._root.data)
        # simply run search from the root
        return self._root.search(data=data, d_parent=d, r=INFINITY, k=k)

    def _init_root(self, data):
        """
        Initializes root of the M-Tree, creates nodes to store the first record
        :param data: initial data
        """
        # (1) Leaf & its ground entry
        # create ground object, add it to a dictionary
        ground = {data: GroundEntry(data=data)}
        # create first leaf, add ground object(s) to it
        leaf = Leaf(entries=ground,
                    data=data,
                    dist_function=self._dist_function,
                    split_function=self.split_function,
                    capacity=self.capacity_max)
        # (2) Router & its routing entry
        # create routing object, add it to a dictionary
        routing = {data: RoutingEntry(subtree=leaf, data=data)}
        # create root, add rooting object(s) to it
        self._root = Root(entries=routing,
                          data=data,
                          dist_function=self._dist_function,
                          split_function=self.split_function,
                          capacity=self.capacity_max,
                          r=INFINITY)

    def _split_root(self):
        """
        Splits root node into two new nodes, which both become subtrees of a new root
        New root will have center in the center of first subtree
        """

        # split all routing entries into two partitions
        partitions = self._root.get_split_data()

        # create new root with 2 new rooting entries
        entries = {}
        # (1) create first router and its routing entry
        router = Router(entries=partitions[0].entries,
                        data=partitions[0].center,
                        dist_function=self._dist_function,
                        split_function=self.split_function,
                        capacity=self.capacity_max,
                        r=partitions[0].r)
        entries[partitions[0].center] = RoutingEntry(subtree=router,
                                                     data=partitions[0].center,
                                                     r=partitions[0].r)
        # (2) create second router and its routing entry
        router = Router(entries=partitions[1].entries,
                        data=partitions[1].center,
                        dist_function=self._dist_function,
                        split_function=self.split_function,
                        capacity=self.capacity_max,
                        r=partitions[1].r)
        parent_dist = self._dist_function(partitions[0].center, partitions[1].center)
        entries[partitions[1].center] = RoutingEntry(subtree=router,
                                                     data=partitions[1].center,
                                                     r=partitions[1].r,
                                                     parent_dist=parent_dist)
        # count distance between centers
        d_centers = self._dist_function(partitions[0].center, partitions[1].center)
        # count root's new radius
        root_r = max(partitions[0].r, d_centers + partitions[1].r)
        # finally create the root
        self._root = Root(entries=entries,
                          data=partitions[0].center,
                          dist_function=self._dist_function,
                          split_function=self.split_function,
                          r=root_r)


In [143]:
mtree = MTree()
mtree.rootNode = MtreeNode()
mtree.rootNode.points = data
mtree.rootNode.height = 1

In [144]:
InputData = np.random.randn(100, 2)


In [145]:
for data in InputData:
    mtree.add(tuple(data))
    mtree.documentMap[tuple(data)] = np.zeros(3, dtype=int)

In [146]:
mtree.knn_query((0,0),3)

[SortableData(data=(0.1115387359346585, 0.03240762525929995), d=0.11615138306903064),
 SortableData(data=(-0.08874665887005521, 0.08653996653908116), d=0.12395618285984462),
 SortableData(data=(0.1363772380903992, 0.034218981309206205), d=0.1406047287647369)]

In [147]:
data

array([ 0.49156461, -0.06946569])

In [148]:
def createLevelMatrix():
    nodes = []
    levelMatrix = {}
    count = 0
    nodeId = 0
    
    for k,v in mtree._root._entries.items():
        #print(v.__str__)
        for  k1,v1 in v.node._entries.items():
            #print(v1)
            node = MtreeNode()
            node.id = nodeId + 1
            node.height = 2
            for  k2,v2 in v1.node._entries.items():
                print(tuple(list(v2.data)))
                mtree.documentMap[tuple(list(v2.data))][2] =  node.id
                node.points.append(list(v2.data))
                count = count + 1
                #print("OOOOOOOOOOOOOOOOOOOOO")
            mtree.rootNode.children.append(node)
            levelMatrix[nodeId] = node
            nodeId = nodeId+1
            #print("*****************")
        #print("________________________")
        return levelMatrix

In [159]:
def min_max_distance(cluster1, cluster2):
    #start = timeit.default_timer()
    #print(cluster1)
    #print(cluster2)
    Z = scipy.spatial.distance.cdist(cluster1, cluster2, 'euclidean')
    #print(Z.min()**2,Z.max()**2)
    #stop = timeit.default_timer()
    #print('Time for mmr: ', stop - start)

    return Z.min()**2,Z.max()**2

In [160]:
levelMatrix = createLevelMatrix()

(0.7684493402473875, -0.7346451743313555)
(-0.47856181702914385, -0.28569302802925073)
(-0.2815550648926676, -1.4929660742606388)
(1.5651207176398418, -0.2448086192317236)
(0.33400833998149587, -1.401350055897268)
(1.123541104727813, -0.38412494340789227)
(0.2295520928923335, -0.8039807539810713)
(0.05438252344933234, 1.0103610829522331)
(1.0432042225307148, 0.10898261840020647)
(-0.14589363841902914, 0.3640159116677471)
(-0.7308719867617345, -0.08148215022071113)
(0.7532823933840085, 1.1640252507554485)
(-1.1371814904168591, 1.4728917660202905)
(-0.49665742354224596, 1.1966387671590324)
(-0.25166316783910164, 0.9945648535877233)
(0.3114616789762348, -0.2505911392359059)
(0.3460798811671935, -0.7776451304128172)
(0.2622597367532192, -0.2376486414638451)
(0.10506611046260253, -0.7144309876763104)
(0.06050445430201904, -0.8220666290974905)
(-0.15581644421290752, -0.38989314664928115)
(-0.1837558148370672, -0.4306246454825666)
(-1.4357586635931654, -0.5887761300689826)
(-1.162047307785820

In [161]:
def createDistanceMatrix(nodes):
    distMatrix = []
    for node1 in range(0,len(levelMatrix)):
        oneMat = []
        for node2 in range(0,len(levelMatrix)):
            dis = min_max_distance(levelMatrix[node1].points,levelMatrix[node2].points)
            oneMat.append(dis)
        distMatrix.append(oneMat)
    return distMatrix

In [162]:
createDistanceMatrix(levelMatrix)

[[(0.0, 4.9681084783363145),
  (0.10536250436582296, 10.425665567927505),
  (0.014272290508959382, 3.093096721460111),
  (0.604562367570917, 10.693817003404202),
  (0.06494198142298277, 16.560330321724273),
  (0.02702351413193557, 10.056612160964407),
  (0.046232476253745845, 6.801641468056579),
  (0.21374031642486313, 16.947747187279727),
  (1.4314854342615, 14.159980120463363)],
 [(0.10536250436582296, 10.425665567927505),
  (0.0, 6.614330020187644),
  (0.421236587579177, 7.264980618640171),
  (0.6818109690563201, 10.36484673969409),
  (0.0093238174609871, 17.369145418832865),
  (0.02141557360626018, 10.709632532626697),
  (0.17744614037840017, 8.448027082846233),
  (0.009711155643028137, 9.204918975832452),
  (0.10185437046145274, 8.976055540893613)],
 [(0.014272290508959382, 3.093096721460111),
  (0.421236587579177, 7.264980618640171),
  (0.0, 0.4022515223426058),
  (1.2389144113533506, 4.46266518706956),
  (0.4811119001187623, 8.652801647890557),
  (0.0882932259059833, 5.633635791